In [1]:
import calendar
import pandas as pd
import numpy as np
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 7, 17)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [3]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,SIS,3600,28.75,0,C28.50,SET,2022-03-10


In [4]:
names = orders['name']
name = names.to_string(index=False)

In [5]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,SIS,1.2000,0.0000,0.0000,0.0000,1.2000,"3,600",2022-03-10,2022-05-13,,0


In [6]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'SIS'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9449,SIS,48.75,25.75,I,0.00,28.75,33.27,1.75,28.50,"3,600",-12,11,"3,600",0,0,C28.50,SET


In [7]:
sql = '''
SELECT trade, name, qty, price, active, reason, market
FROM orders
WHERE trade = 'B'
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,B,AIT,18000,4.20,0,52WL,SET
1,B,AP,10000,9.50,0,MinP,SET100
2,B,ASK,3000,30.00,0,52WL,SET
3,B,BANPU,12000,11.50,0,1L,SET50
4,B,BCPG,9000,10.00,0,52WL,SET100
5,B,IVL,3000,43.50,0,13WL,SET50
6,B,KKP,3000,60.00,0,52WL,SET100
7,B,KTC,2100,52.75,0,52WL,SET50
8,B,MCS,5000,10.60,0,52WL,SET
9,B,RBF,9000,13.50,0,52WL,SET100


In [8]:
names = df_buy['name'] 
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AIT', 'AP', 'ASK', 'BANPU', 'BCPG', 'IVL', 'KKP', 'KTC', 'MCS', 'RBF', 'RCL', 'SINGER', 'SNC', 'SSP', 'TMT'"

In [9]:
sql = """
SELECT name, market, price, max_price, min_price FROM stocks WHERE name IN (%s)
"""
sql = sql % in_p
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict)


SELECT name, market, price, max_price, min_price FROM stocks WHERE name IN ('AIT', 'AP', 'ASK', 'BANPU', 'BCPG', 'IVL', 'KKP', 'KTC', 'MCS', 'RBF', 'RCL', 'SINGER', 'SNC', 'SSP', 'TMT')



,name,market,price,max_price,min_price
0,AIT,sSET,5.55,9.80,4.24
1,AP,SET100 / SETHD,9.25,12.00,7.45
2,ASK,SET,30.00,49.50,28.25
3,BANPU,SET50 / SETCLMV / SETTHSI,12.50,14.90,9.65
4,BCPG,SET100 / SETCLMV / SETHD / SETTHSI,10.20,15.40,10.20
5,IVL,SET50 / SETTHSI,43.75,52.75,36.00
6,KKP,SET100 / SETCLMV / SETHD / SETTHSI,60.75,76.25,49.75
7,KTC,SET50 / SETTHSI,54.25,70.00,52.75
8,MCS,sSET,11.00,16.20,10.80
9,RBF,SET100 / SETCLMV / SETWB,13.00,24.50,12.80


In [10]:
df_merge = pd.merge(df_buy, df_stocks, on='name', how='inner')
df_merge

,trade,name,qty,price_x,active,reason,market_x,market_y,price_y,max_price,min_price
0,B,AIT,18000,4.20,0,52WL,SET,sSET,5.55,9.80,4.24
1,B,AP,10000,9.50,0,MinP,SET100,SET100 / SETHD,9.25,12.00,7.45
2,B,ASK,3000,30.00,0,52WL,SET,SET,30.00,49.50,28.25
3,B,BANPU,12000,11.50,0,1L,SET50,SET50 / SETCLMV / SETTHSI,12.50,14.90,9.65
4,B,BCPG,9000,10.00,0,52WL,SET100,SET100 / SETCLMV / SETHD / SETTHSI,10.20,15.40,10.20
5,B,IVL,3000,43.50,0,13WL,SET50,SET50 / SETTHSI,43.75,52.75,36.00
6,B,KKP,3000,60.00,0,52WL,SET100,SET100 / SETCLMV / SETHD / SETTHSI,60.75,76.25,49.75
7,B,KTC,2100,52.75,0,52WL,SET50,SET50 / SETTHSI,54.25,70.00,52.75
8,B,MCS,5000,10.60,0,52WL,SET,sSET,11.00,16.20,10.80
9,B,RBF,9000,13.50,0,52WL,SET100,SET100 / SETCLMV / SETWB,13.00,24.50,12.80


In [11]:
# Include both having diviedend and having no dividend records
mask = (df_merge['price_x'] <= df_merge['min_price'])      
df_52wl = df_merge[mask].copy()
df_52wl['reason'] = '52WL'
df_52wl

,trade,name,qty,price_x,active,reason,market_x,market_y,price_y,max_price,min_price
0,B,AIT,18000,4.20,0,52WL,SET,sSET,5.55,9.80,4.24
4,B,BCPG,9000,10.00,0,52WL,SET100,SET100 / SETCLMV / SETHD / SETTHSI,10.20,15.40,10.20
7,B,KTC,2100,52.75,0,52WL,SET50,SET50 / SETTHSI,54.25,70.00,52.75
8,B,MCS,5000,10.60,0,52WL,SET,sSET,11.00,16.20,10.80
11,B,SINGER,3000,35.50,0,52WL,SET100,SET100 / SETWB,38.00,59.25,35.50
14,B,TMT,12000,7.90,0,52WL,SET,sSET / SETTHSI,8.05,12.00,7.95


In [12]:
names = df_52wl['name'] 
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AIT', 'BCPG', 'KTC', 'MCS', 'SINGER', 'TMT'"

In [13]:
sqlUpd = """
UPDATE orders
SET reason = '52WL'
WHERE name IN (%s)"""
sqlUpd = sqlUpd % in_p
rp = conlite.execute(sqlUpd)
rp.rowcount

6

In [14]:
pd.read_sql_query('SELECT * FROM orders WHERE reason = "52WL"', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
1,3,B,TMT,12000,7.90,0,52WL,SET,2022-08-27
2,4,B,AIT,18000,4.20,0,52WL,SET,2022-02-02
3,6,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
4,8,B,BCPG,9000,10.00,0,52WL,SET100,2022-02-02
5,10,B,KKP,3000,60.00,0,52WL,SET100,2022-04-28
6,11,B,KTC,2100,52.75,0,52WL,SET50,2022-04-18
7,12,B,RBF,9000,13.50,0,52WL,SET100,2022-02-02
8,13,B,SINGER,3000,35.50,0,52WL,SET100,2022-02-02
9,15,B,SSP,10000,8.75,0,52WL,SET,2022-02-02


In [15]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
1,2,B,RCL,3000,35.25,0,RD20pct,SET100,2022-08-27
2,3,B,TMT,12000,7.90,0,52WL,SET,2022-08-27
3,4,B,AIT,18000,4.20,0,52WL,SET,2022-02-02
4,5,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
5,6,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
6,7,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
7,8,B,BCPG,9000,10.00,0,52WL,SET100,2022-02-02
8,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
9,10,B,KKP,3000,60.00,0,52WL,SET100,2022-04-28


In [16]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)